## Imports

In [32]:
import sys
import random
from pathlib import Path
from collections import Counter

import datasets
from datasets import load_dataset_builder, load_dataset

from utils import replace_text_with_whitespace

In [33]:
ELK_PATH = Path("../../../elk/")
ELK_PATH.resolve()

PosixPath('/fsx/home-augustas/elk')

In [34]:
modules = [
    ELK_PATH,
    ELK_PATH / "elk" / "promptsource",
]

for module in modules:
    if not str(module) in sys.path:
        sys.path.insert(0, str(module.resolve()))

sys.path[:2]

['/fsx/home-augustas/elk/elk/promptsource', '/fsx/home-augustas/elk']

In [35]:
from templates import DatasetTemplates

In [36]:
# Disable the logging of the datasets library
import datasets

datasets.logging.set_verbosity_error()

## Config

In [37]:
# all datasets used in Burns et al. (2022) (apart from story_cloze)
# BURNS_DATASETS = [
#     "ag_news",
#     "amazon_polarity",
#     "dbpedia_14",
#     "glue:qnli",
#     "imdb",
#     "piqa",
#     "super_glue:boolq",
#     "super_glue:copa",
#     "super_glue:rte",
# ]
BURNS_DATASETS = ["ag_news"]

SPLIT = "validation"

# These numbers are chosen so that both datasets have
# approximately 10k examples in total
N_PER_DATASET = 1200 if SPLIT == "train" else 1800
N_PER_DATASET = 1200 if SPLIT == "train" else 7600
# if SPLIT == "train":
#     assert N_PER_DATASET <= 2490, "N_PER_DATASET must be <= 2490 for train"
# else:
#     assert N_PER_DATASET <= 1838, "N_PER_DATASET must be <= 1838 for validation"

## Load and inspect the datasets

In [38]:
dataset_dict = {}
for dataset_path in BURNS_DATASETS:
    print(dataset_path)

    # Parse dataset name
    dataset_name = None    
    if ":" in dataset_path:
        dataset_path, dataset_name = dataset_path.split(":")
    
    
    # Get the most validation-like split
    available_splits = load_dataset_builder(
        dataset_path, name=dataset_name
    ).info.splits.keys()
    split = "validation" if "validation" in available_splits else "test"
    split = split if SPLIT != "train" else "train"
    print(f"{split=}")

    # Load the dataset
    dataset = load_dataset(
        dataset_path, name=dataset_name, split=split,
    )

    # Get a desired subset of the data
    n = N_PER_DATASET if dataset.num_rows > N_PER_DATASET else dataset.num_rows
    dataset = dataset.shuffle(seed=42).select(range(n))

    print(dataset.num_rows)

    key = f"{dataset_path}/{dataset_name}" if dataset_name else dataset_path
    dataset_dict[key] = dataset

    print("-----------------------------------")

ag_news


split='test'
7600
-----------------------------------


In [39]:
assert all([len(Counter(dataset["label"])) > 1 for dataset in dataset_dict.values()])

In [40]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {dataset.num_rows}")

ag_news: 7600


In [41]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {len(Counter(dataset['label']))}")

ag_news: 4


## Split into validation and test

In [42]:
ppo_dataset_dict = {}
test_dataset_dict = {}

for dataset_name, dataset in dataset_dict.items():
    print(dataset_name)
    splits = dataset.train_test_split(test_size=0.1315, seed=42)
    # print(splits)
    print(splits["train"].num_rows, splits["test"].num_rows)

    ppo_dataset_dict[dataset_name] = splits["train"]
    test_dataset_dict[dataset_name] = splits["test"]
    print("-----------------------------------")

ag_news
6600 1000
-----------------------------------


In [43]:
sum(dataset.num_rows for dataset in ppo_dataset_dict.values()), sum(dataset.num_rows for dataset in test_dataset_dict.values())

(6600, 1000)

In [44]:
# datasets.DatasetDict(ppo_dataset_dict).save_to_disk("datasets/burns_datasets_VINC_ag_news_ppo_training_raw")

In [45]:
# datasets.DatasetDict(test_dataset_dict).save_to_disk("datasets/burns_datasets_VINC_ag_news_test_raw")

## Get the templates

In [46]:
# Set dataset_dict to test_dataset
dataset_dict = datasets.DatasetDict.load_from_disk("datasets/burns_datasets_VINC_ag_news_test_raw")
dataset_dict

DatasetDict({
    ag_news: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

In [47]:
sum(dataset.num_rows for dataset in dataset_dict.values())

1000

In [48]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {len(Counter(dataset['label']))}")

ag_news: 4


In [49]:
dataset_template_dict = {}

for dataset_path in dataset_dict.keys():
    dataset_templates = DatasetTemplates(dataset_path)

    dataset_templates.templates = {
        x.name: x for x in dataset_templates.templates.values() if x.get_answer_choices_list(dataset_dict[dataset_path][0]) is not None
    }

    dataset_template_dict[dataset_path] = dataset_templates

In [50]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

ag_news: 15


## Inspect the prompts

In [51]:
# dataset_name = "ag_news"

# for template_name, template in dataset_template_dict[dataset_name].templates.items():
#     # print(template_name)
#     # print(dataset_dict[dataset_name][0])
#     q, a = template.apply(
#         dataset_dict[dataset_name][0]
#     )
#     # print(q == q.strip())
#     # print(a == a.strip())
#     # print(" ".join([q, a.strip()]))
#     print(" ".join([q, a]))
#     print("---------------------------------")

## Form the dataset for the chosen split

In [52]:
# for dataset_name, dataset_templates in dataset_template_dict.items():
#     print(dataset_name)
#     for template_name, template in dataset_templates.templates.items():
#         print(f"{template_name}: {template.get_fixed_answer_choices_list()}")

#     print("---------------------------------")


In [53]:
# for dataset_name, dataset_templates in dataset_template_dict.items():
#     print(dataset_name)
#     for template_name, template in dataset_templates.templates.items():
#         print(f"{template_name}: {template.get_answer_choices_list(dataset_dict[dataset_name][0])}")

#     print("---------------------------------")

In [54]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

ag_news: 15


In [55]:
%%time

# Reproducibility
random.seed(2023)

ALLOWED_KEYS = ["text", "label", "original_dataset", "template_name"]

new_dataset = []

for dataset_name, dataset in dataset_dict.items():
    print(dataset_name)

    # if dataset_name != "ag_news": continue
    # if dataset_name != "imdb": continue
    # if dataset_name != "piqa": continue

    for idx, entry in enumerate(dataset):
        new_entry = entry.copy()
        
        # In case we need to know which dataset the entry came from
        new_entry["original_dataset"] = dataset_name

        # Sample a random template
        template_name = random.choice(
            list(dataset_template_dict[dataset_name].templates.keys())
        )
        template = dataset_template_dict[dataset_name].templates[template_name]
        new_entry["template_name"] = template_name

        # Whether the sample will be truthful or not
        is_truthful = random.choice([True, False])

        # If not truthful, sample a random incorrect label
        if not is_truthful and len(dataset.features["label"].names) > 2:
            label_mappping = dataset.features["label"] # [class 1, class 2, ...]
            all_label_ids = [label_mappping.str2int(x) for x in label_mappping.names] # [0, 1, ...]
            incorrect_label_id = all_label_ids[(1 - new_entry["label"]) % len(all_label_ids)]
            incorrect_label = template.get_fixed_answer_choices_list()[incorrect_label_id]
            
        
        # Apply the template
        if is_truthful:
            new_text = " ".join(template.apply(new_entry))
        elif not is_truthful and len(dataset.features["label"].names) > 2:
            q, a = template.apply(new_entry)
            incorrect_label = replace_text_with_whitespace(a, incorrect_label)
            new_text = " ".join([q, incorrect_label])
        else:
            # Untruthful binary case
            new_entry["label"] = 1 - new_entry["label"]
            new_text = " ".join(template.apply(new_entry))

        new_entry["text"] = new_text

        # We can now change the label to whether the sample is truthful or not
        new_entry["label"] = int(is_truthful)

        # Remove all other keys
        new_entry = {k: v for k, v in new_entry.items() if k in ALLOWED_KEYS}

        # Append to the new dataset
        new_dataset.append(new_entry)

ag_news


CPU times: user 2.05 s, sys: 1.31 ms, total: 2.05 s
Wall time: 2.05 s


In [56]:
my_dataset = datasets.Dataset.from_list(new_dataset)
my_dataset

Dataset({
    features: ['text', 'label', 'original_dataset', 'template_name'],
    num_rows: 1000
})

In [57]:
my_dataset[0]

{'text': "Consider the following example: ''' IMF sees rising oil prices having little impact on global growth TOKYO: Rising oil prices are unlikely to deal a major blow to global economic growth although the trend may seem  quot;uncomfortable, quot; a researcher with the International Monetary Fund says. ''' Choice 1: Business. Choice 2: Science and technology.Between choice 1 and choice 2, what is the topic of this example? Science and technology",
 'label': 0,
 'original_dataset': 'ag_news',
 'template_name': 'burns_2'}

In [58]:
Counter(my_dataset["label"]), Counter(my_dataset["original_dataset"])

(Counter({1: 512, 0: 488}), Counter({'ag_news': 1000}))

In [60]:
# my_dataset.to_parquet(f"datasets/burns_datasets_VINC_ag_news_validation.parquet")

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 397.94ba/s]


390186

In [61]:
!ls -lah datasets

total 312K
drwxr-xr-x 4 augustas Domain Users  33K Jun 19 16:07 .
drwxr-xr-x 4 augustas Domain Users  33K Jun 19 15:59 ..
drwxr-xr-x 3 augustas Domain Users  25K Jun 19 16:07 burns_datasets_VINC_ag_news_ppo_training_raw
drwxr-xr-x 3 augustas Domain Users  25K Jun 19 15:59 burns_datasets_VINC_ag_news_test_raw
-rw-r--r-- 1 augustas Domain Users 188K Jun 19 16:00 burns_datasets_VINC_ag_news_train.parquet
-rw-r--r-- 1 augustas Domain Users 187K Jun 19 16:12 burns_datasets_VINC_ag_news_validation.parquet
